In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
import polars as pl
import joblib
import kaggle_evaluation.mcts_inference_server

In [2]:
def create_agent_map(row):
    selection_map = {
        'UCB1': 1,
        'UCB1GRAVE': 2,
        'ProgressiveHistory': 3,
        'UCB1Tuned': 4
    }
    
    playout_map = {
        'Random200': 1,
        'MAST': 2,
        'NST': 3
    }

    parts = row.split('-')

    selection = parts[1]
    exploration_const = float(parts[2])
    playout = parts[3]
    score_bounds = parts[4] == 'true'

    # Convert categorical values to numeric values
    selection_value = selection_map.get(selection)
    playout_value = playout_map.get(playout)
    score_bounds_value = 1 if score_bounds else 0

    # Create the embedding map
    return [selection_value, exploration_const, playout_value, score_bounds_value]

def data_preprocess(test_df):
    test_features = test_df.drop(columns=['Id', 'GameRulesetName', 'agent1', 'agent2', 'EnglishRules', 'LudRules'])
    
    test_features = test_features.fillna(test_features.mean())  # Fill NaNs with column means
    test_features = test_features.fillna(0)  # Fill remaining NaNs with 0
    test_features = test_features.replace([np.inf, -np.inf], np.nan)  # Replace infinities with NaN
    test_features = test_features.fillna(0)  # Handle infinities turned into NaNs
    assert test_features.isna().sum().sum() == 0, "NaN values still present after fillna."
    
    # Load scaler and pca model
    scaler = joblib.load('/kaggle/input/scaler_pca/scikitlearn/default/1/scaler.pkl')
    pca = joblib.load('/kaggle/input/scaler_pca/scikitlearn/default/1/pca_model.pkl')
    
    # Fit test_df to scaler and pca
    test_features_scaled = scaler.transform(test_features)
    test_features_pca = pca.transform(test_features_scaled)
    test_pca_df = pd.DataFrame(test_features_pca, columns=[f"pca_{i+1}" for i in range(test_features_pca.shape[1])])
    
    # Categorize MCTS
    agents_test = test_df[['agent1', 'agent2']]
    pca_df_test = pd.concat([agents_test, test_pca_df], axis=1)
    pca_df_test[['agent1_emb1', 'agent1_emb2', 'agent1_emb3', 'agent1_emb4']] = pd.DataFrame(
    agents_test['agent1'].apply(create_agent_map).tolist(), index=pca_df_test.index
    )
    pca_df_test[['agent2_emb1', 'agent2_emb2', 'agent2_emb3', 'agent2_emb4']] = pd.DataFrame(
        agents_test['agent2'].apply(create_agent_map).tolist(), index=pca_df_test.index
    )
    pca_df_test = pca_df_test.drop(columns=['agent1', 'agent2'])
    
    return pca_df_test

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    test_df = test.to_pandas()
    
    # Data pre-process
    pca_df_test = data_preprocess(test_df)
    test_data = xgb.DMatrix(pca_df_test)
    
    # Load model
    model = xgb.Booster()
    model.load_model('/kaggle/input/xgboost_pca/scikitlearn/default/1/xgboost_model_fulldata.json')
    
    # Make predictions
    predictions = model.predict(test_data)

    sample_sub = sample_sub.with_columns(pl.col('utility_agent1') + predictions)
    
    return sample_sub

In [3]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

/tmp/ipykernel_17/1875400978.py:33: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_features = test_features.fillna(test_features.mean())  # Fill NaNs with column means
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.5.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For